# Tool Binding

Tool Binding is the step where you register tools with a Language Model (LLM) so that:

 - The LLM knows what tools are available

 - It knows what each tool does (via description)

 - It knows what input format to use (via schema)

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
import requests

from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# Tool create

@tool
def multiply(a:int, b:int) -> int:
    """Given Two numbers a and b this tool returns product"""
    return a*b

In [8]:
# Tool Binding
llm = ChatOpenAI()

In [9]:
llm_with_tools = llm.bind_tools([multiply])

# Tool Calling

Tool calling is the process where the LLM decides, during a conversation or task, taht it needs to use a specific tool (function) - and generates a structured output with:
 
 - the name of tool
 - and the argumants to call it with

The LLM doesn't run the tool - it just suggest the tool and the input argumants. The actual execution is handled by the Langchain or you 

In [10]:
llm_with_tools.invoke("Hi how are you").content

"Hello! I'm here and ready to assist you. How can I help you today?"

In [11]:
result = llm_with_tools.invoke("can you multiple 3 with 10")

In [12]:
from pprint import pprint
pprint(result)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4ScgPVnv14ZkGq9egJ6ACyY5', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSOj9MtyFmmIWZp3S6NlyjWpazYrI', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cdfe8f19-aebd-458b-b42b-83c1c7f4ecf2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_4ScgPVnv14ZkGq9egJ6ACyY5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

Now you Notice the above results you'll notice that ``content`` is empty, but in ``additional_kwargs`` we got a new property named ``tool_calls``

Another important thing to note here is that, even though LLM has mentioned the tool we bind with it, it really doesn't executes it and give reults from it. That's because LLM doesn't really executes the tool, **the tool must be called by Langcahin or you**

# Tool Excecution

Tool Execution is the step where where the actual python function (tool) i srun using the input argumnets that the LLM suggested during tool calling

In simppler words:
 🧠 The LLM says:
 
  - "Hey, call the ``multiply`` tool with a=8 and b=7"

  ⚙️ **Tool Execution** is when you or Langchain actually run :
  
  - `multiply(a=8, b=7)` 
    
    and get the results : 56


In [13]:
results = llm_with_tools.invoke('Can you multply 3 and 10')

In [14]:
pprint(results)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Qv2R1WBUezPePqdJ4vbbqexk', 'function': {'arguments': '{"a": 3, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 61, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSOjAmnJ67ZXwxSRIti0oHt9mBxwZ', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-704c2bb0-44b1-4f07-af52-cba04f1c0f1d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_Qv2R1WBUezPePqdJ4vbbqexk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 33, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {

In [15]:
results.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_Qv2R1WBUezPePqdJ4vbbqexk',
 'type': 'tool_call'}

In [16]:
multiply.invoke(results.tool_calls[0])

ToolMessage(content='30', name='multiply', tool_call_id='call_Qv2R1WBUezPePqdJ4vbbqexk')

These were the individual components, lets build a complete pipeline

In [17]:
# Build LLM
llm = ChatOpenAI()

# Bind Tool with LLM
llm_with_tools = llm.bind_tools([multiply])

In [18]:
query = HumanMessage('Can you multiply 3 with 10')
messages = [query]

In [19]:
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wdSsWAHD20BXbvLBOOddOCVM', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSOjCQ0jGlq6ZEywQ0IXbFBbm9cvH', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ebcb1295-dce1-41dc-bcca-c8f2a8752685-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_wdSsWAHD20BXbvLBOOddOCVM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

Append this ``AIMessage`` to messages array

In [20]:
messages.append(result)
messages

[HumanMessage(content='Can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wdSsWAHD20BXbvLBOOddOCVM', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSOjCQ0jGlq6ZEywQ0IXbFBbm9cvH', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ebcb1295-dce1-41dc-bcca-c8f2a8752685-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_wdSsWAHD20BXbvLBOOddOCVM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'tot

In [21]:
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='30', name='multiply', tool_call_id='call_wdSsWAHD20BXbvLBOOddOCVM')

Append this ``ToolMessage`` to the messages array

In [22]:
messages.append(tool_result)
messages

[HumanMessage(content='Can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wdSsWAHD20BXbvLBOOddOCVM', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSOjCQ0jGlq6ZEywQ0IXbFBbm9cvH', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ebcb1295-dce1-41dc-bcca-c8f2a8752685-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_wdSsWAHD20BXbvLBOOddOCVM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'tot

You can very well understand what we are doing here. We are actually **maintaining the history of our conversations**, as we usually do.

Now lets again call our llm, and this time we'll pass our conversation history(messages array) to model 

In [23]:
_result = llm_with_tools.invoke(messages)

In [24]:
_result

AIMessage(content='The product of 3 and 10 is 30.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 85, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSOjDuzIO9a0nt67fpu2u6y48KDdQ', 'finish_reason': 'stop', 'logprobs': None}, id='run-af803e20-fd3b-495f-94e3-046b94074a2b-0', usage_metadata={'input_tokens': 85, 'output_tokens': 14, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
_result.content 

'The product of 3 and 10 is 30.'

### Drumroll Please!

This is our first tool calling. Now you can ask thi smodl any query and it will use our tool.

# Currency Conversion Tool

We'll build a currency conversion tool. Where our main motive is to get the current rates of Pakistani Currency using the the ExcahgeRate API.

In [26]:
#  Tool create


@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency.
    """
    url = f"https://v6.exchangerate-api.com/v6/13c68535f9d92af500b7a7ef/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()


@tool
def convert(base_currency: int, conversion_rate: float) -> float:
    """
    Given a currency conversion rate this function calcaulate the target currency value from a given base currency value
    """
    return base_currency * conversion_rate

In [27]:
# create LLM

llm = ChatOpenAI()

# bind the tools
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [28]:
question = HumanMessage("What is the conversion factor between USD and PKR, and based on that can you convert 10 usd to PKR")

In [29]:
messages = []
messages.append(question)

In [30]:
ai_message = llm_with_tools.invoke(messages)

In [31]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': 'call_FauWXP31BZpkt9f7a7Ieh7Lt',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 10, 'conversion_rate': 160},
  'id': 'call_qd6BPHBL1EGMVKk6ATU7Veqg',
  'type': 'tool_call'}]

As you can see from these reults, this time, two tools have been called.

But observe this Keenly, and you'll ntice some anomly here in the second tool. The LLM has called the second tool with some old values, instead of values from ``get_conversion_factor`` tool.

### Why does this happened?

The issue arises because the LLM does not automatically chain the outputs of one tool to the inputs of another. In our case:

The LLM called the get_conversion_factor tool to fetch the conversion rate between USD and PKR.
However, when calling the convert tool, it used an old or hardcoded conversion rate (162.7) instead of dynamically using the output from the get_conversion_factor tool.
This happens because the LLM does not have the capability to execute tools or manage dependencies between tool outputs and inputs. It only suggests tool calls based on its training and context.

Why this happened:
 - Lack of state management: The LLM does not maintain a state or memory of the output from the first tool call (get_conversion_factor) to use it as input for the second tool call (convert).
Static reasoning: The LLM generates tool calls independently, based on the prompt and context, without dynamically linking the tools' outputs and inputs.


So, now we will rewrite our tools, where the `conversion_rate` in second argument will depend on the the output of first value 

In [49]:
#  Tool create
from typing import Annotated
from langchain_core.tools import InjectedToolArg

@tool
def sec_get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency.
    """
    url = f"https://v6.exchangerate-api.com/v6/13c68535f9d92af500b7a7ef/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()


@tool
def sec_convert(base_currency: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function calcaulate the target currency value from a given base currency value
    """
    return base_currency * conversion_rate

# Understanding `InjectedToolArg` in LangChain

The `InjectedToolArg` annotation is a special feature in LangChain that helps solve the tool chaining problem we encountered earlier. Here's how it works:

### Purpose
- It tells LangChain that the `conversion_rate` parameter should be automatically filled with a value from a previous tool's output
- This creates an automatic dependency between tools, allowing them to pass data between each other

### Syntax Breakdown


In [33]:
conversion_rate: Annotated[float, InjectedToolArg]

- `Annotated`: A Python type hint that allows adding metadata to type annotations
- `float`: The expected type of the parameter
- `InjectedToolArg`: Indicates this argument should be automatically injected from another tool's output

### How it Works
1. When the first tool (`get_conversion_factor`) is called, it returns conversion data
2. LangChain automatically extracts the relevant value from that response
3. When calling the `convert` tool, LangChain automatically injects the extracted value into the `conversion_rate` parameter

### Benefits
- Eliminates the need for manual tool output handling
- Prevents the LLM from using hardcoded/outdated values
- Creates a more reliable tool chain where data flows automatically between tools




This is much more reliable than the previous version where the LLM was trying to guess or use hardcoded conversion rates.

In [57]:
sec_llm = ChatOpenAI()
sec_llm_with_tools = sec_llm.bind_tools([sec_get_conversion_factor, sec_convert])

In [59]:
messages = []
sec_human_message = HumanMessage("What is the conversion factor between USD and PKR, and based on that can you convert 10 usd to PKR")
messages.append(sec_human_message)

In [60]:
sec_ai_message = sec_llm_with_tools.invoke(messages)

In [61]:
messages.append(sec_ai_message)

In [62]:
sec_ai_message.tool_calls

[{'name': 'sec_get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': 'call_ypobxJPpxJauicB69gMxY8ug',
  'type': 'tool_call'},
 {'name': 'sec_convert',
  'args': {'base_currency': 10},
  'id': 'call_jFXMAybhl8m7JYJ1D22OG0d9',
  'type': 'tool_call'}]

In [63]:
import json
for tool_call in sec_ai_message.tool_calls:

    # Execute the first tool and get the value of conversion rate
    if tool_call['name'] == "sec_get_conversion_factor":
        tool_message_1 = sec_get_conversion_factor.invoke(tool_call)

        #Fetch the conversion rate
        conversion_rate = json.loads(tool_message_1.content)['conversion_rate']
        
        # Append this tool message to message list
        messages.append(tool_message_1)
        
    # Execute the second tool using the conversion rate from tool 1
    if tool_call['name'] =='sec_convert':
        
        #fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message_2 = convert.invoke(tool_call)
        messages.append(tool_message_2)

In [64]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 usd to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ypobxJPpxJauicB69gMxY8ug', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "PKR"}', 'name': 'sec_get_conversion_factor'}, 'type': 'function'}, {'id': 'call_jFXMAybhl8m7JYJ1D22OG0d9', 'function': {'arguments': '{"base_currency": 10}', 'name': 'sec_convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 126, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSP54bKYQuLtT49vUWRBkfoBg22FN', 'finish_reason': 't

In [66]:
sec_llm_with_tools.invoke(messages).content

'The conversion factor between USD and PKR is 280.9017. \n\nBased on this conversion factor, 10 USD is equal to 2809.017 PKR.'